dependencies

In [ ]:
import pandas
import matplotlib.pyplot as plt
import datetime
import os

set working directory<br>
put all downloaded files into this working directory. <br>
new created spatial files will be stored into your home geodatabase.

In [ ]:
os.chdir(r"C:\Users\Brian\Desktop\ResaleNew\WorkingFolder")

download resale flat prices csv files and put them in the working directory<br>
https://beta.data.gov.sg/datasets/189/view

concatenate all files<br>
file names

In [ ]:
file_names = [r"Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv", 
              r"Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv", 
              r"Resale Flat Prices (Based on Registration Date), From 2012 to Dec 2014.csv", 
              r"Resale Flat Prices (Based on Registration Date), From 2015 to Dec 2016.csv", 
              r"Resale Flat Prices (Based on Registration Date), From 2017 onwards.csv"]

initialize an empty DataFrame to store the combined data

In [ ]:
data_frames = []

loop through each file and append data to the combined DataFrame

In [ ]:
for file_name in file_names:
    df = pandas.read_csv(file_name)
    data_frames.append(df)

concatenate

In [ ]:
all_addresses = pandas.concat(data_frames, ignore_index=True)
    
# preview
pandas.set_option("display.max_columns", None)
print(all_addresses)
pandas.reset_option("display.max_columns")

clean out irrelevant columns

In [ ]:
selected_columns = [
    "month",
    "town",
    "flat_type",
    "street_name",
    "storey_range",
    "floor_area_sqm",
    "flat_model",
    "lease_commence_date",
    "resale_price",
    "address"
]

In [ ]:
all_addresses_clean = all_addresses[selected_columns]

preview

In [ ]:
pandas.set_option("display.max_columns", None)
print(all_addresses_clean)
pandas.reset_option("display.max_columns")

filter for Bishan from "town" column

In [ ]:
bishan_addresses = all_addresses_clean[all_addresses_clean["town"] == "BISHAN"]

preview

In [ ]:
pandas.set_option("display.max_columns", None)
print(bishan_addresses)
pandas.reset_option("display.max_columns")

save as dataframe

In [ ]:
bishan_addresses_df = bishan_addresses.copy()

calculate price_per_sqm

In [ ]:
bishan_addresses_df["price_per_sqm"] = bishan_addresses_df["resale_price"] / bishan_addresses_df["floor_area_sqm"]

calculate remaining lease

In [ ]:
bishan_addresses_df["remaining_lease"] = 99 - (datetime.date.today().year - bishan_addresses_df["lease_commence_date"])

calculate price_per_remaining_lease

In [ ]:
bishan_addresses_df["price_per_remaining_lease"] = bishan_addresses_df["resale_price"] / bishan_addresses_df["remaining_lease"]

calculate price per sq m per remaining lease (PPSMPRL) or pomperly

In [ ]:
bishan_addresses_df["PPSMPRL"] = bishan_addresses_df["resale_price"] / bishan_addresses_df["floor_area_sqm"] / bishan_addresses_df["remaining_lease"]

preview

In [ ]:
pandas.set_option("display.max_columns", None)
print(bishan_addresses_df)
pandas.reset_option("display.max_columns")

save as dataframe

In [ ]:
bishan_resale = bishan_addresses_df.copy()

export as csv

In [ ]:
output_file_path = "bishan_resale.csv"
bishan_resale.to_csv(output_file_path, index=False)
print(f"DataFrame exported to {output_file_path}")

floor area vs resale price

In [ ]:
x_axis = "floor_area_sqm"
y_axis = "resale_price"

In [ ]:
plt.scatter(bishan_addresses_df[x_axis], bishan_addresses_df[y_axis], alpha=0.1)
plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.title(f"{x_axis} VS {y_axis}")
plt.show()

remaining lease vs resale price

In [ ]:
x_axis = "remaining_lease"
y_axis = "resale_price"

floor area VS price per sq m

In [ ]:
plt.scatter(bishan_addresses_df[x_axis], bishan_addresses_df[y_axis], alpha=0.1)
plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.title(f"{x_axis} VS {y_axis}")
plt.show()

remaining lease vs resale price

In [ ]:
x_axis = "PPSMPRRL"
y_axis = "resale_price"

floor area VS price per sq m

In [ ]:
plt.scatter(bishan_addresses_df[x_axis], bishan_addresses_df[y_axis], alpha=0.1)
plt.xlabel(x_axis)
plt.ylabel(y_axis)
plt.title(f"{x_axis} VS {y_axis}")
plt.show()

filter for only unique addresses in bishan

In [ ]:
unique_bishan_addresses = bishan_addresses.drop_duplicates(subset=["address"])

preview with all columns

In [ ]:
pandas.set_option("display.max_columns", None)
print(unique_bishan_addresses)
pandas.reset_option("display.max_columns")

export as CSV file

In [ ]:
output_file_path = r"unique_bishan_addresses.csv"
unique_bishan_addresses.to_csv(output_file_path, index=False)
print(f"DataFrame exported to '{output_file_path}' sucessfully")

Geocode unique addresses<br>
This has been commented out so that it does not consume my arcgis credits when rerunning :)<br>
arcpy.geocoding.GeocodeAddresses(

In [ ]:
    in_table=r"unique_bishan_addresses.csv",
    address_locator="https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/ArcGIS World Geocoding Service",
    in_address_fields="'Single Line Input' address VISIBLE NONE",
    out_feature_class=r"unique_bishan_addresses",
    out_relationship_type="STATIC",
    country="SG",
    location_type="ADDRESS_LOCATION",
    category=None,
    output_fields="ALL"
)

download files and put them in the working directory

Singapore buildings shapefile from <br>
https://download.bbbike.org/osm/bbbike/Singapore/

Singapore master plan 2014 planning area boundary from <br>
https://data.gov.sg/dataset/master-plan-2014-planning-area-boundary-web

repair geometry of buildings file

In [ ]:
arcpy.management.RepairGeometry(
    in_features=r"Singapore-shp\shape\buildings.shp",
    delete_null="DELETE_NULL",
    validation_method="ESRI"
)

In [ ]:
print("Success")

extract bishan boundary from master plan 2014 planning area

In [ ]:
arcpy.conversion.ExportFeatures(
    in_features=r"master-plan-2014-planning-area-boundary-web-shp\MP14_PLNG_AREA_WEB_PL.shp",
    out_features=r"bishan_boundary",
    where_clause="PLN_AREA_N = 'BISHAN'",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping='',
    sort_field=None
)

In [ ]:
print("Success")

clip buildings with bishan boundary

In [ ]:
arcpy.analysis.Clip(
    in_features=r"Singapore-shp\shape\buildings.shp",
    clip_features="bishan_boundary",
    out_feature_class="bishan_buildings",
    cluster_tolerance=None
)

In [ ]:
print("Success")

inspect shapefile, and add features where buildings are missing<br>
when that is done, export data as new feature

In [ ]:
arcpy.conversion.ExportFeatures(
    in_features=r"bishan_buildings",
    out_features=r"bishan_buildings_fixed",
    where_clause="",
    use_field_alias_as_name="NOT_USE_ALIAS",
    field_mapping="",
    sort_field=None
)

In [ ]:
print("Success")

calculate average resale price per address

In [ ]:
bishan_resale_price = bishan_resale.groupby("address")["resale_price"].mean()

calculate average resale price per address

In [ ]:
bishan_price_per_sqm = bishan_resale.groupby("address")["price_per_sqm"].mean()

calculate average price per remaining lease

In [ ]:
bishan_price_per_remaining_lease = bishan_resale.groupby("address")["price_per_remaining_lease"].mean()

calculate average PPSMPRRL

In [ ]:
bishan_ppsmprl = bishan_resale.groupby("address")["PPSMPRL"].mean()

Merge

In [ ]:
merged_df = pandas.merge(bishan_resale_price, bishan_price_per_sqm, on="address")
merged_df = pandas.merge(merged_df, bishan_price_per_remaining_lease, on="address")
merged_df = pandas.merge(merged_df, bishan_ppsmprl, on="address")

Display the merged DataFrame

In [ ]:
pandas.set_option("display.max_columns", None)
print(merged_df)
pandas.reset_option("display.max_columns")

ave as dataframe

In [ ]:
bishan_averages_df = merged_df.reset_index().copy()

export as csv

In [ ]:
output_file_path = r"bishan_averages.csv"
bishan_averages_df.to_csv(output_file_path, index=False)
print(f"DataFrame exported to '{output_file_path}' successfully.")

permanently join with bishan_avg_price.csv column

In [ ]:
arcpy.management.JoinField(
    in_data="unique_bishan_addresses",
    in_field="USER_address",
    join_table="bishan_averages.csv",
    join_field="address",
#     fields="resale_price;price_per_sqm;price_per_remaining_lease;PPSMPRL",
    fm_option="NOT_USE_FM",
    field_mapping=None
)

In [ ]:
print("Success")

spatial join point addresses to buildings within 10m

In [ ]:
arcpy.analysis.SpatialJoin(
    target_features="bishan_buildings",
    join_features="unique_bishan_addresses",
    out_feature_class="bishan_buildings_SpatialJoin",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="WITHIN_A_DISTANCE",
    search_radius="10 Meters",
    distance_field_name=""
)

In [ ]:
print("Success")